# check oof df

In [1]:
import os
import gc
import yaml
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("/kaggle/src/dss_utils")
sys.path.append("/kaggle/src/exp")
sys.path.append("/kaggle/src/model")
sys.path.append("/kaggle/src/data")
from dss_metrics import score
from training_loop import get_valid_values_dict, concat_valid_input_info, get_oof_df, get_key_df
from dss_model import get_model
from dss_dataloader import get_loader


In [2]:
output_dir = "/kaggle/working"
# exp_name = "exp017_inputtargettd_flip_epoch10"
# exp_name = "exp018_inputtargetd_hour_notflip_epoch10"
# exp_name = "exp020_dense_chh_epoch10"
# exp_name = "exp020_dense_chh_skffold_epoch30"
# exp_name = "exp021_dense_chh_skffold_removesomeseries_epoch30"
exp_name = "exp023_alldata_skf_dense_epoch30"
folds = [0, 1, 2, 3, 4]
# folds = [0]
oof_df = pd.DataFrame()
for i in folds:
    print("fold", i)
    # df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"raw_oof_df_fold{i}.parquet"))
    df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"oof_df_fold{i}.parquet"))
    oof_df = pd.concat([oof_df, df], axis=0)


print(len(oof_df["series_id"].unique()))   


fold 0
fold 1
fold 2
fold 3
fold 4
277


In [3]:
oof_df = oof_df.sort_values(["series_id", "step"])
display(oof_df)


,series_id,step,anglez,enmo,second,date,anglez_mean_36,anglez_std_36,anglez_mean_60,anglez_std_60,...,event_onset,event_wakeup,series_date_key,series_date_key_str,fold,event_pred,class_pred,class_target,class_det_pred,class_det_target
0,038441c925bb,0.0,2.636700,0.0217,0,2018-08-14,0.0,0.0,0.0,0.0,...,0,0,0,038441c925bb_2018-08-14,2,0.162433,0.170782,0.0,0.708279,1.0
1,038441c925bb,1.0,2.636800,0.0215,5,2018-08-14,0.0,0.0,0.0,0.0,...,0,0,0,038441c925bb_2018-08-14,2,0.000000,0.063509,0.0,0.796173,1.0
2,038441c925bb,2.0,2.637000,0.0216,10,2018-08-14,0.0,0.0,0.0,0.0,...,0,0,0,038441c925bb_2018-08-14,2,0.000000,0.037930,0.0,0.834294,1.0
3,038441c925bb,3.0,2.636800,0.0213,15,2018-08-14,0.0,0.0,0.0,0.0,...,0,0,0,038441c925bb_2018-08-14,2,0.000000,0.036773,0.0,0.838677,1.0
4,038441c925bb,4.0,2.636800,0.0215,20,2018-08-14,0.0,0.0,0.0,0.0,...,0,0,0,038441c925bb_2018-08-14,2,0.000000,0.038418,0.0,0.839958,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25500432,fe90110788d2,592375.0,-27.277500,0.0204,35,2017-09-08,0.0,0.0,0.0,0.0,...,0,0,7745,fe90110788d2_2017-09-08,3,0.000000,0.039315,-1.0,0.310502,0.0
25500433,fe90110788d2,592376.0,-27.032499,0.0233,40,2017-09-08,0.0,0.0,0.0,0.0,...,0,0,7745,fe90110788d2_2017-09-08,3,0.000000,0.038247,-1.0,0.307564,0.0
25500434,fe90110788d2,592377.0,-26.841200,0.0202,45,2017-09-08,0.0,0.0,0.0,0.0,...,0,0,7745,fe90110788d2_2017-09-08,3,0.000000,0.036800,-1.0,0.303194,0.0
25500435,fe90110788d2,592378.0,-26.723900,0.0199,50,2017-09-08,0.0,0.0,0.0,0.0,...,0,0,7745,fe90110788d2_2017-09-08,3,0.000000,0.034940,-1.0,0.297748,0.0


In [4]:
# postprocess_fn
# series_idでgroupbyして、class_predに対して対象の列のデータから前のN個の列までのデータの平均をとる
import torch
import torch.nn as nn


def postprocess_notrolling(df):
    df = df.copy()
    df = df[df["second"]==0].reset_index(drop=True)
    # 一つ前のclass_predを取得
    df["class_pred_before"] = df.groupby("series_id")["class_pred"].shift(1)
    # 一つ後のclass_predを取得
    df["class_pred_after"] = df.groupby("series_id")["class_pred"].shift(-1)
    df["event_pred"] = df["class_pred_before"] - df["class_pred_after"]
    not_predicted_mask = (df["class_pred"] != -1).astype(int)
    df["event_pred"] = df["event_pred"] * not_predicted_mask
    df = df.drop(["class_pred_before", "class_pred_after"], axis=1)
    return df

# 1step 0.5secで30minなら60*30=1800step
def postprocess_downsample_fn(df, N=3, maxpool_kernel_size=3, maxpool_stride=1):
    df = df.copy()
    df = df[df["second"]==0].reset_index(drop=True)
    df["class_pred_beforemean"] = df.groupby("series_id")["class_pred"].apply(lambda x: x.rolling(N, min_periods=1).mean())
    df["class_pred_aftermean"] = df.groupby("series_id")["class_pred"].apply(lambda x: x[::-1].rolling(N, min_periods=1).mean()[::-1])
    df["event_pred"] = df["class_pred_beforemean"] - df["class_pred_aftermean"]
    not_predicted_mask = (df["class_pred"] != -1).astype(int)
    df["event_pred"] = df["event_pred"] * not_predicted_mask

    # 入力サイズと出力サイズが一致するようにpaddingを調整
    maxpool_padding = int((maxpool_kernel_size - maxpool_stride) / 2)
    # maxpoolしてピーク検出
    max_pooling = nn.MaxPool1d(maxpool_kernel_size, stride=maxpool_stride, padding=maxpool_padding)
    event_pred = df["event_pred"].values
    event_pred = torch.tensor(event_pred).unsqueeze(0)
    pooled_event_pred = max_pooling(np.abs(event_pred)).squeeze(0).numpy()
    event_pred = event_pred.squeeze(0).numpy()
    # peakのところだけ残すmaskを作成
    peak_event_pred_mask = np.where(pooled_event_pred == np.abs(event_pred), 1, 0)
    peak_event_pred = event_pred * peak_event_pred_mask
    df["event_pred"] = peak_event_pred
    df = df.drop(["class_pred_beforemean", "class_pred_aftermean"], axis=1)
    return df

def postprocess_downsample_notmaxpool(df, N=11):
    df = df.copy()
    df["class_pred_beforemean"] = df.groupby("series_id")["class_pred"].apply(lambda x: x.rolling(N, min_periods=1).mean())
    df["class_pred_aftermean"] = df.groupby("series_id")["class_pred"].apply(lambda x: x[::-1].rolling(N, min_periods=1).mean()[::-1])
    df["event_pred"] = df["class_pred_beforemean"] - df["class_pred_aftermean"]
    df = df.drop(["class_pred_beforemean", "class_pred_aftermean"], axis=1)
    return df

def make_submission_df(df, threshold=0.1):
    df = df[["series_id", "step", "event_pred"]].copy()
    # thresholdより大きいときは1,-thresholdより小さいときは-1,それ以外は0
    df["event"] = df["event_pred"].apply(lambda x: 1 if x > threshold else -1 if x < -threshold else 0)
    df = df[df["event"] != 0].copy()
    df["event"] = df["event"].replace({1: "wakeup", -1: "onset"})
    df["score"] = df["event_pred"].apply(lambda x: np.clip(np.abs(x), 0.0, 1.0))
    return df


In [5]:
train_event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
train_event_df = train_event_df[train_event_df["step"].notnull()].copy()
# train_event_df = train_event_df[train_event_df["series_id"].isin(oof_df["series_id"].unique())].copy()
train_event_df = train_event_df.reset_index(drop=True)
display(train_event_df.head())


,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


In [6]:
df = postprocess_notrolling(oof_df)
sub_df = make_submission_df(df, threshold=0.1)
display(sub_df.head())


,series_id,step,event_pred,event,score
269,038441c925bb,3228.0,-0.176746,onset,0.176746
271,038441c925bb,3252.0,0.149304,wakeup,0.149304
413,038441c925bb,4956.0,-0.108625,onset,0.108625
414,038441c925bb,4968.0,-0.246692,onset,0.246692
415,038441c925bb,4980.0,-0.305853,onset,0.305853


In [7]:
oof_notroll_score = score(train_event_df, sub_df)
print(oof_notroll_score)


0.3555177362997367


In [8]:
# average_size = 31
# max_kernel_size = 31
# df = postprocess_downsample_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size)
# sub_df = make_submission_df(df, threshold=0.1)
# ave_maxpool_score = score(train_event_df, sub_df)
# print(ave_maxpool_score)


In [9]:
%%time
max_kernel_size = 3
tmp_score = 0
for average_size_ in range(3, 21, 2):
    print("average_size", average_size_, "max_kernel_size", max_kernel_size)
    df = postprocess_downsample_fn(oof_df, N=average_size_, maxpool_kernel_size=max_kernel_size)
    sub_df = make_submission_df(df, threshold=0.1)
    if len(sub_df) == 0:
        print("event not detected")
        continue
    score_ = score(train_event_df, sub_df)
    print(score_)
    if tmp_score > score_:
        break
    tmp_score = score_
    average_size = average_size_


average_size 3 max_kernel_size 3
0.5915479598831237
average_size 5 max_kernel_size 3


KeyboardInterrupt: 

In [ ]:
tmp_score = 0
max_kernel_size_ = max_kernel_size
for max_kernel_size in range(3, 31, 2):
    print("average_size", average_size, "max_kernel_size", max_kernel_size)
    df = postprocess_downsample_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size)
    sub_df = make_submission_df(df, threshold=0.1)
    if len(sub_df) == 0:
        print("event not detected")
        continue
    score_ = score(train_event_df, sub_df)
    print(score_)
    if tmp_score > score_:
        break
    tmp_score = score_
    max_kernel_size_ = max_kernel_size


average_size 13 max_kernel_size 3
0.7329384430633479
average_size 13 max_kernel_size 5
0.7327516596172834


In [ ]:
df = postprocess_downsample_fn(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size_)
sub_df = make_submission_df(df, threshold=0.01)
print(average_size, max_kernel_size_)
print(len(sub_df))
if len(sub_df) == 0:
    print("event not detected")
    score_ = 0
else:
    score_ = score(train_event_df, sub_df)
print(score_)


13 3
31642
0.7459116079636723


"exp012_targetdownsample_epoch10"


average_size 9 max_kernel_size 3
0.7310038029952944

thr=0.01 -> 0.7469626949452008

---

exp_name = "exp012_targetdownsample_epoch5"

average_size 11 max_kernel_size 5
0.7325993414687024

0.7462916305317064

---
exp017_inputtargettd_flip_epoch10


0.7574596100410679

---
exp020_dense_chh_epoch10

0.7544748020752414

---
exp020_dense_chh_skffold_epoch30

0.747115174909606
